In [1]:
using Revise

In [2]:
using ModelVerification

In [3]:
using Flux
using LazySets

In [5]:
small_nnet_file = "../test/networks/small_nnet.nnet"
# small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
small_nnet = ModelVerification.read_nnet(small_nnet_file)
model = Flux.Chain(small_nnet)
input = Hyperrectangle(low = [-2.5], high = [2.5])
output = Hyperrectangle(low = [30.0], high = [60.0])
info = nothing
prop_method = Ai2z()
search_method = BFS(max_iter=100, batch_size=1)
split_method = Bisect(1)
problem = Problem(model, input, output)
verify(search_method, split_method, prop_method, problem)


batch_reach
Any[Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([0.0], [2.5])]
Vector{Any}
batch_reach
Zonotope{Float64, Vector{Float64}, Matrix{Float64}}[Zonotope{Float64, Vector{Float64}, Matrix{Float64}}([1.5, 1.5], [2.5; 2.5;;])]
Vector{Zonotope{Float64, Vector{Float64}, Matrix{Float64}}}
batch_reach
Zonotope{Float64, Vector{Float64}, Matrix{Float64}}[Zonotope{Float64, Vector{Float64}, Matrix{Float64}}([8.5, 8.5], [10.0; 10.0;;])]
Vector{Zonotope{Float64, Vector{Float64}, Matrix{Float64}}}
sub_branches


Tuple{Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}, Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}, Nothing}

[(Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([-1.25], [1.25]), Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([45.0], [15.0]), nothing), (Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([1.25], [1.25]), Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([45.0], [15.0]), nothing)]
batch_reach
Any[Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([-1.25], [1.25])]
Vector{Any}
batch_reach
Zonotope{Float64, Vector{Float64}, Matrix{Float64}}[Zonotope{Float64, Vector{Float64}, Matrix{Float64}}([0.25, 0.25], [1.25; 1.25;;])]
Vector{Zonotope{Float64, Vector{Float64}, Matrix{Float64}}}
batch_reach
Zonotope{Float64, Vector{Float64}, Matrix{Float64}}[Zonotope{Float64, Vector{Float64}, Matrix{Float64}}([3.5, 3.5], [5.0; 5.0;;])]
Vector{Zonotope{Float64, Vector{Float64}, Matrix{Float64}}}


CounterExampleResult(:violated, [-1.25])